# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [7]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [8]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/" 

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [11]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load the page.
    time.sleep(2)

        # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height
     
#     提示：可參考以下的Stack Overflow: 
#     https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
#     '''

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\user\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 工商協進會理事長林伯豐：基本工資’若調漲　本外勞薪資應脫鉤

▲工商協進會理事長林伯豐。（圖／記者陳政錄攝）
記者余弦妙／台北報導
勞動部上個月底已召開基本工資工作小組，就是要討論接下來第三季的基本工資審議委員會召開時間，對於今年基本工資是否要調漲，工商協進會理事長林伯豐今（8）日表示，整體經濟不好的情況下，若要調漲基本薪資，本外勞薪資就應該脫鉤。
對於基本工資是否調漲，林伯豐指出，以目前的經濟景氣非常低迷的情況來看，且失業率也持續增加的情況之下，呼籲政府應該要好好思考，在目前這樣的情況之下，基本工資是否要調漲，因為一旦調漲基本工資，企業成本在現在大環境不好的情況下，有可能會為了成本減少人力，屆時可能失業率也會增加。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[1] 微星召開臨時董事會　由創始元老黃金請接任新總經理

▲微星科技。（圖／翻攝微星）
記者姚惠茹／台北報導
微星科技（2377）今（8）日舉行臨時董事會，會議中決議由副董事長暨資深副總裁黃金請接任總經理，即日起生效，
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[2] 迎A股牛市　投信：A50投資買點到

▲上海證券交易所。（圖／達志圖庫）
記者陳心怡／台北報導
A股近期持續上漲，上證指數從疫情後2646點的低點，反彈至3332點，漲幅達25.9%，７月以來則上漲達11.23%，市場上漸漸有了「牛市已至」的聲音，國泰投信認為，從評價面與資金來看，Ａ股具備長期投資價值。
國泰中國A50正2(00655L)經理人游日傑表示，近期陸股火熱，首先來自於市場信心回籠，其中，中國金融股評價面相對偏低，且今年以來漲幅落後，在市場信心回籠下，帶動金融股補漲行情，不論證券、銀行或保險類股漲幅皆相當驚人。
此外，中國6月財新PMI數據持續全面回升，其中服務業PMI更上升至58.4，創2

▲三倍券超商預購上路。（圖／記者周宸亘攝）
記者林淑慧／台北報導
振興三倍券預購今（8）日開放第八天，依據經濟部統計，截至今天下午4時為止，上官網申購的民眾已突破1001萬5000人次；為鼓勵民眾申購，經濟部祭出超值包及電動機車等大獎，希望能拉高數位綁定的人數。
依據經濟部統計，目前紙本預購共857萬9256人，透過官網預購者有52萬2750人，選擇四大超商預購的有805萬6506人；至於數位綁定共143萬5744人，含信用卡97萬4446人，電子票證17萬6694人，行動支付28萬4604人。
振興三倍券即將在7月15日上路，傳統市場及夜市也積極響應，經濟部中部辦公室即推出「振興三倍券超值包好康！再抽市集抵用券！」活動，活動期間民眾至140個參與活動市集消費，憑振興三倍券(含電子票券、行動支付)至攤位購買超值包或超值套餐後，就可獲得抽獎序號卡1份，參與市集抽獎，可抽出2000元市集抵用券，名額6,300名，預估可帶動市集4億元的營業額。
官員指出，「振興三倍券超值包好康！再抽市集抵用券！」活動，市集包含傳統市場及夜市，將於7月15日開跑至8月31日止，預計將於8月11日(二)、8月25日(二)及9月8日(二)，辦理3場次抽獎，每次抽出2,100名獎項，持續炒熱市集買氣。
經濟部中部辦公室表示，為吸引振興三倍券消費，各家紛紛提出「3倍券變4倍」、「3倍券變5倍」等方案，在市集，用振興三倍券200元購買超值包，就有機會抽到2,000元市集抵用券，讓「3倍券變10倍」。
 
►過期票券別浪費！簡單3步驟　教你搶救妙招
 
----------------------------------------------------------------------

[7] 新台幣收盤貶值2.6分　收在29.606元兌1美元	

▲新台幣收盤收29.606元。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（8）日收盤收29.606元，貶值2.6分，成交金額7.92億美元。
----------------------------------------------------------------------

[8] 台積電擴大採用再生能源！簽署1.2GW購電契約　碳排放量「年減218.9萬公噸」

▲台積電宣布將擴大採用再生能源。圖為台灣離

▲工商協進會理事長林伯豐(左)、桃園市長鄭文燦(右)。（圖／記者余弦妙攝）
記者余弦妙／台北報導
工商協進會今（8）日邀請桃園市長鄭文燦分享桃園防疫紓困過程，工商協進會理事長林伯豐表示，台灣疫情雖然已經趨緩，政府雖陸續推出振興紓困措施，但整體經濟復甦仍未恢復到一定水準，因此工商界期盼政府可加大紓困振興力道。
雖然台灣超前部署得宜，因此疫情相比全球來說較為穩定，不過，林伯豐也提出隱憂，目前全球各國大多還是封城情況，台灣又是以外貿為導向的經濟體，可想而知台灣經濟一定會受到影響。
林伯豐說，雖然政府陸續推出紓困振興措施，但國內外需求仍未恢復正常，景氣復甦緩慢，因此工商業也向政府呼籲希望可以加大紓困措施振興力道，對於上市櫃公司的正常融資，應積極給予支持以帶動經濟，中小企業紓困融資，則應政策明確、程序透明、手續更簡化。
另外，林伯豐也說，也需要加速與各國政府協商邊境開放事宣，建議可採平等互惠原則，並採用相同的檢疫標準，另優先開放商務人士往來，缩短或取消居家隔離天數。
對於商務人士是否可以解封，鄭文燦表示，目前台灣對於外國人的留居相關政策都不太友善，自己去了解過後也確實是如此，台灣確實在經商的友善度真的不夠，中央目前也很努力地去談平等互惠原則，不過細節都還是需要討論，但他認為大原則上商務人士確實是要列為第一階段恢復的重點。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[16] 裕國董事會決議向前董座楊連發行使歸入權　9/23再次召開股東會

▲裕國冷凍於6月24日股東會爆發推擠衝突，董事會決議將在9月23日再次召開股東常會。（圖／CTWANT）
圖文／CTWANT
裕國冷凍冷藏公司（8905）本年度股東常會於6月24日流會後，董事長楊育偉緊急召集董事會，於7月7日召開且全體董監事均到齊，會中決議將於9月23日召集股東常會，且公告相關議案。依據裕國重大訊息公告，股東常會議案除沿用前次公告內容外，董事會一併決議擬依公司法第209條第5項規定，提請股東會向前董座楊連發行使歸入權。
裕國公司發出聲明表示，楊連發擔任裕國集團董事長兼總經理期間，同時開設食品製造公司楊名國際，並兼任該公司負責人，其與裕國皆為相同業務之公司，應依公司法第209條第1項

▲台股可望挑戰30年前歷史新高。（圖／記者湯興漢攝，資料照）
記者楊絡懸／台北報導
台股行情下半年正式啟動。群益投信表示，由於資金環境仍相當寬鬆，台股流動性風險低與政策面偏多，短線來看，台股創下波段新高後將挑戰歷史新高1990年2月10日收盤價12495.34點，或是1990年2月12日盤中最高點12682.41點。目前指數站上各期均線、且各期均線皆呈現走揚態勢，有利多方後續啟動攻勢。
群益投信表示，台股今年來表現是亞股中僅次於陸股，顯示台股的投資價值逐漸被內外法人認同，評價不貴又有殖利率。台股在低油價、低通膨，以及寬鬆貨幣政策下的資金挹注，對股市都有正面助益，股市仍可望往正向發展。
展望2020下半年，蘋果新產品更是將帶動供應鏈業績的焦點，台灣科技廠商一直是蘋果主要供應鏈，因此蘋果商品發表對於台股後市有一定的影響，從歷史經驗來看，在新機發表前相關供應鏈啟動拉貨潮，一直到新機發表後，對於台股股價多有正面助益。
▲群益創新科技基金經理人朱翡勵。（圖／記者楊絡懸攝）
群益創新科技基金經理人朱翡勵表示，從基本面、資金面、價值面、信心面四大面向來看，寬鬆貨幣政策搭配各國振興方案，全球景氣自谷底翻揚，展望逐季向上，基本面上沒太大問題；而全球央行力行寬鬆，台商回流資金、外資認錯回補買盤，挹注金融市場活水，也顯示出資金的活躍有利台股後市表現。
此外，台股殖利率穩定，廠商獲利顯著回升，評價具有上修空間，深具投資價值；最後今年面臨科技大創新，創新科技翻轉未來新生活，因此可特別著重創新科技類股。
朱翡勵進一步指出，產業部分，持續看好績優且有展望的成長型公司表現，特別是產業趨勢佳、具新科技和新應用題材之類股，創新科技如5G、AI、物聯網應用興起，新規格、新平台帶動下一波科技革命，相關供應鏈類股值得留意。
在傳產方面，朱翡勵透露，特別看好新興生活的運動業、老樹新開的紡織業、精益求精的工具機和厚積薄發的生技業，這些都是今年下半年台股投資不可忽視的主要趨勢。
▲群益投信「2020下半年」台股看法。（圖／記者楊絡懸製表）
【延伸閱讀】
----------------------------------------------------------------------

[27] 換美元衝一波！台銀換匯價殺到29.6字頭　換10萬多賺近3300元

▲美元換匯價持續創低。（圖／記者湯興漢攝